In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from pprint import pprint
from tensorflow import keras

In [2]:
# read dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = np.expand_dims(x_train, axis=-1) / 255. # (# of samples, 28, 28, 1)
x_test = np.expand_dims(x_test, axis=-1) / 255.

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
def build_model():
    model = keras.models.Sequential([
        keras.layers.InputLayer((28, 28, 1)),
        keras.layers.Conv2D(16, 3, activation='relu'),
        keras.layers.Conv2D(16, 3, activation='relu'),
        keras.layers.MaxPooling2D(),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(10),
        keras.layers.Softmax(),
    ])
    return model

In [4]:
# backup initial weights
model = build_model()
PATH = 'init.h5'
model.save_weights(PATH)

In [5]:
K = 5
EPOCHS = 1
BATCH_SIZE = 256

from sklearn.model_selection import KFold
kfold = KFold(n_splits=K)

fold_losses = []
fold_accs = []

for fold_i, (train_ids, val_ids) in enumerate(kfold.split(x_train)):
    print(f'train size: {len(train_ids)}, val size: {len(val_ids)}')
    # load init weights
    model.load_weights(PATH)
    
    model.compile(keras.optimizers.Adam(), 
                  keras.losses.SparseCategoricalCrossentropy(),
                  metrics=[keras.metrics.SparseCategoricalAccuracy(name='acc')])
    
    # use ids from KFold to select train and val data
    model.fit(x_train[train_ids], y_train[train_ids],
              batch_size=BATCH_SIZE,
              epochs=EPOCHS, 
              validation_data=(x_train[val_ids], y_train[val_ids]), 
              verbose=0)
    
    # Evaluate
    test_loss, test_acc = model.evaluate(x_test, y_test, 
                                         batch_size=BATCH_SIZE, 
                                         verbose=0)

    print(f'Fold {fold_i}, test acc: {test_acc:.3f}')
    
    fold_losses.append(test_loss)
    fold_accs.append(test_acc)

train size: 48000, val size: 12000
Fold 0, test acc: 0.270
train size: 48000, val size: 12000
Fold 1, test acc: 0.259
train size: 48000, val size: 12000
Fold 2, test acc: 0.234
train size: 48000, val size: 12000
Fold 3, test acc: 0.276
train size: 48000, val size: 12000
Fold 4, test acc: 0.302


In [6]:
print(f"Loss: mean {np.mean(fold_losses):.3f}, std: {np.std(fold_losses):.3f}")
print(f"Acc: mean {np.mean(fold_accs):.3f}, std: {np.std(fold_accs):.3f}")

Loss: mean 2.035, std: 0.006
Acc: mean 0.268, std: 0.022
